In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import json

# California Data

In [2]:
#data collected from https://www.cde.ca.gov/ds/sd/sd/filesabd.asp
#https://www.oregon.gov/ode/reports-and-data/students/Pages/Student-Enrollment-Reports.aspx

url= "http://dq.cde.ca.gov/dataquest/dlfile/dlfile.aspx?cLevel=School&cYear=2019-20&cCat=Enrollment&cPage=filesenr.asp"
ca_enrollment_data = pd.read_table(url)
ca_enrollment_data


,CDS_CODE,COUNTY,DISTRICT,SCHOOL,ETHNIC,GENDER,KDGN,GR_1,GR_2,GR_3,...,GR_7,GR_8,UNGR_ELM,GR_9,GR_10,GR_11,GR_12,UNGR_SEC,ENR_TOTAL,ADULT
0,7617966119515,Contra Costa,West Contra Costa Unified,Hercules Middle,6,F,0,0,0,0,...,15,17,0,0,0,0,0,0,48,0
1,7617966119515,Contra Costa,West Contra Costa Unified,Hercules Middle,1,M,0,0,0,0,...,1,0,0,0,0,0,0,0,2,0
2,7617966119515,Contra Costa,West Contra Costa Unified,Hercules Middle,3,F,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
3,7617966119515,Contra Costa,West Contra Costa Unified,Hercules Middle,4,F,0,0,0,0,...,15,23,0,0,0,0,0,0,67,0
4,7617966119515,Contra Costa,West Contra Costa Unified,Hercules Middle,5,F,0,0,0,0,...,24,26,0,0,0,0,0,0,80,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131611,37681303732625,San Diego,Grossmont Union High,Grossmont High,7,M,0,0,0,0,...,0,0,0,141,138,111,120,0,510,0
131612,37681890000001,San Diego,Lakeside Union Elementary,"Nonpublic, Nonsectarian Schools",6,F,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
131613,37681890000001,San Diego,Lakeside Union Elementary,"Nonpublic, Nonsectarian Schools",7,F,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
131614,37681890000001,San Diego,Lakeside Union Elementary,"Nonpublic, Nonsectarian Schools",5,M,0,0,0,0,...,0,1,0,0,0,0,0,0,2,0


In [3]:

#modify demo to include ethnicity as columns
ca_enrollment_race = ca_enrollment_data.pivot_table(values='ENR_TOTAL', index='COUNTY', columns='ETHNIC', aggfunc='sum')
ca_enrollment_race.head()

ETHNIC,0,1,2,3,4,5,6,7,9
COUNTY,,,,,,,,,
Alameda,2410.0,597.0,60548.0,2276.0,10324.0,77433.0,20865.0,39186.0,13692.0
Alpine,1.0,36.0,NaN,NaN,NaN,2.0,NaN,26.0,5.0
Amador,32.0,99.0,20.0,8.0,20.0,925.0,20.0,2765.0,277.0
Butte,432.0,690.0,1777.0,100.0,124.0,7469.0,602.0,17002.0,1838.0
Calaveras,5.0,82.0,31.0,5.0,17.0,1128.0,39.0,3671.0,337.0


In [4]:
#rename the ethnicity columns
ca_enrollment_race_2 = ca_enrollment_race.rename(columns = {0:'OTHER',
                                    1:'AMERICAN_INDIAN',
                                    2:'ASIAN',
                                    3:'PACIFIC_ISLANDER',
                                    4:'FILIPINO',
                                    5:'LATINO',
                                    6:'AFRICAN_AMERICAN',
                                    7:'WHITE',
                                    9:'MULTI_RACE'})
ca_enrollment_race_2.reset_index(inplace=True)
ca_enrollment_race_2.head()

ETHNIC,COUNTY,OTHER,AMERICAN_INDIAN,ASIAN,PACIFIC_ISLANDER,FILIPINO,LATINO,AFRICAN_AMERICAN,WHITE,MULTI_RACE
0,Alameda,2410.0,597.0,60548.0,2276.0,10324.0,77433.0,20865.0,39186.0,13692.0
1,Alpine,1.0,36.0,NaN,NaN,NaN,2.0,NaN,26.0,5.0
2,Amador,32.0,99.0,20.0,8.0,20.0,925.0,20.0,2765.0,277.0
3,Butte,432.0,690.0,1777.0,100.0,124.0,7469.0,602.0,17002.0,1838.0
4,Calaveras,5.0,82.0,31.0,5.0,17.0,1128.0,39.0,3671.0,337.0


In [32]:

ca_enrollment_race_2['OTHER_TOTAL'] = ca_enrollment_race_2['OTHER'] + ca_enrollment_race_2['MULTI_RACE']
ca_enrollment_race_2['ASIAN_TOTAL'] = ca_enrollment_race_2['ASIAN'] + ca_enrollment_race_2['FILIPINO']
ca_enrollment_race_2.head(1)
ca_clean = ca_enrollment_race_2[['COUNTY', 'AMERICAN_INDIAN', 'PACIFIC_ISLANDER', 'LATINO','AFRICAN_AMERICAN','WHITE','OTHER_TOTAL','ASIANS_TOTAL']].copy()

#replace NAN with zero
ca_clean.fillna(0,inplace = True)
ca_clean.head()

# Add column of CA
ca_clean['STATE'] = "CA"
ca_clean

ca_clean.rename(
  columns={
    'ASIANS_TOTAL' : 'ASIAN',
    'OTHER_TOTAL' : 'OTHER/MULTI'
  },
  inplace=True
)
ca_clean

ca_clean.reset_index(inplace=False)
#ca_clean.drop(["ETHNIC"], axis=1)


ETHNIC,index,COUNTY,AMERICAN_INDIAN,PACIFIC_ISLANDER,LATINO,AFRICAN_AMERICAN,WHITE,OTHER/MULTI,ASIAN,STATE
0,0,Alameda,597.0,2276.0,77433.0,20865.0,39186.0,16102.0,70872.0,CA
1,1,Alpine,36.0,0.0,2.0,0.0,26.0,6.0,0.0,CA
2,2,Amador,99.0,8.0,925.0,20.0,2765.0,309.0,40.0,CA
3,3,Butte,690.0,100.0,7469.0,602.0,17002.0,2270.0,1901.0,CA
4,4,Calaveras,82.0,5.0,1128.0,39.0,3671.0,342.0,48.0,CA
5,5,Colusa,55.0,9.0,3749.0,33.0,780.0,49.0,55.0,CA
6,6,Contra Costa,534.0,1100.0,65518.0,15329.0,51141.0,14070.0,30714.0,CA
7,7,Del Norte,613.0,3.0,942.0,25.0,2228.0,0.0,211.0,CA
8,8,El Dorado,240.0,78.0,6260.0,350.0,20759.0,1852.0,1587.0,CA
9,9,Fresno,1199.0,480.0,135997.0,10046.0,34222.0,5048.0,20866.0,CA


In [36]:
list(ca_clean.columns)

['COUNTY',
 'AMERICAN_INDIAN',
 'PACIFIC_ISLANDER',
 'LATINO',
 'AFRICAN_AMERICAN',
 'WHITE',
 'OTHER/MULTI',
 'ASIAN',
 'STATE']

# Oregon Data

In [49]:
url= "https://www.oregon.gov/ode/reports-and-data/students/Documents/fallmembershipreport_20192020.xlsx"

or_enrollment_data = pd.read_excel(url, sheet_name = "District (19-20)")
or_enrollment_data
list(or_enrollment_data.columns)
or_clean = or_enrollment_data[['County', 
                   '2019-20 American Indian/Alaska Native ', 
                   '2019-20 Asian ', 
                   '2019-20 Native Hawaiian/ Pacific Islander', 
                   '2019-20 Black/African American ', 
                   '2019-20 Hispanic/ Latino', 
                   '2019-20 White ', 
                   '2019-20 Multiracial ']].copy()
or_clean

,County,2019-20 American Indian/Alaska Native,2019-20 Asian,2019-20 Native Hawaiian/ Pacific Islander,2019-20 Black/African American,2019-20 Hispanic/ Latino,2019-20 White,2019-20 Multiracial
0,Lake,2,0,0,0,0,5,0
1,Malheur,1,1,0,0,60,227,3
2,Benton,5,18,0,2,28,253,15
3,Yamhill,12,6,0,3,110,652,41
4,Malheur,0,0,0,0,25,65,4
...,...,...,...,...,...,...,...,...
205,Yamhill,210,2,5,1,83,522,53
206,Douglas,26,2,4,5,106,1166,84
207,Marion,6,22,13,13,4685,839,50
208,Yamhill,1,5,1,8,94,897,46


In [50]:
or_clean['STATE'] = "OR"
or_clean

,County,2019-20 American Indian/Alaska Native,2019-20 Asian,2019-20 Native Hawaiian/ Pacific Islander,2019-20 Black/African American,2019-20 Hispanic/ Latino,2019-20 White,2019-20 Multiracial,STATE
0,Lake,2,0,0,0,0,5,0,OR
1,Malheur,1,1,0,0,60,227,3,OR
2,Benton,5,18,0,2,28,253,15,OR
3,Yamhill,12,6,0,3,110,652,41,OR
4,Malheur,0,0,0,0,25,65,4,OR
...,...,...,...,...,...,...,...,...,...
205,Yamhill,210,2,5,1,83,522,53,OR
206,Douglas,26,2,4,5,106,1166,84,OR
207,Marion,6,22,13,13,4685,839,50,OR
208,Yamhill,1,5,1,8,94,897,46,OR


In [51]:
or_clean.rename(
  columns={'County':'COUNTY',
 '2019-20 American Indian/Alaska Native ':'AMERICAN_INDIAN',
 '2019-20 Native Hawaiian/ Pacific Islander':'PACIFIC_ISLANDER',
 '2019-20 Hispanic/ Latino':'LATINO',
 '2019-20 Black/African American ':'AFRICAN_AMERICAN',
 '2019-20 White ':'WHITE',
 '2019-20 Multiracial ':'OTHER/MULTI',
 '2019-20 Asian ':'ASIAN',
  },
  inplace=True
)
or_clean

,COUNTY,AMERICAN_INDIAN,ASIAN,PACIFIC_ISLANDER,AFRICAN_AMERICAN,LATINO,WHITE,OTHER/MULTI,STATE
0,Lake,2,0,0,0,0,5,0,OR
1,Malheur,1,1,0,0,60,227,3,OR
2,Benton,5,18,0,2,28,253,15,OR
3,Yamhill,12,6,0,3,110,652,41,OR
4,Malheur,0,0,0,0,25,65,4,OR
...,...,...,...,...,...,...,...,...,...
205,Yamhill,210,2,5,1,83,522,53,OR
206,Douglas,26,2,4,5,106,1166,84,OR
207,Marion,6,22,13,13,4685,839,50,OR
208,Yamhill,1,5,1,8,94,897,46,OR


In [54]:
json_file = "https://data.wa.gov/resource/gtd3-scga.json"
wa_enrollment_data = pd.read_json(json_file)
wa_enrollment_data.head(100)

,schoolyear,organizationlevel,county,esdname,districtname,schoolname,gradelevel,all_students,female,gender_x,...,non_mobile,non_section_504,students_without_disabilities,dataasof,esdorganizationid,districtcode,districtorganizationid,schoolcode,schoolorganizationid,currentschooltype
0,2019-20,State,Multiple,State Total,State Total,State Total,10th Grade,84690,41154.0,139.0,...,82499.0,79726,74529.0,2020-02-11T00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-20,State,Multiple,State Total,State Total,State Total,11th Grade,83731,40877.0,153.0,...,81156.0,78730,74017.0,2020-02-11T00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-20,State,Multiple,State Total,State Total,State Total,12th Grade,91585,44319.0,194.0,...,86412.0,86395,79242.0,2020-02-11T00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-20,State,Multiple,State Total,State Total,State Total,1st Grade,83765,40675.0,21.0,...,81966.0,82713,73240.0,2020-02-11T00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-20,State,Multiple,State Total,State Total,State Total,2nd Grade,83362,40569.0,29.0,...,81697.0,81768,72221.0,2020-02-11T00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2019-20,School,Lewis,Capital Region ESD 113,Adna School District,Adna Elementary School,Pre-Kindergarten,3,2.0,NaN,...,3.0,3,NaN,2020-02-11T00:00:00.000,100004.0,21226.0,100011.0,2227.0,101834.0,P
96,2019-20,School,Lewis,Capital Region ESD 113,Adna School District,Adna Middle/High School,10th Grade,56,32.0,NaN,...,56.0,54,49.0,2020-02-11T00:00:00.000,100004.0,21226.0,100011.0,2441.0,101835.0,P
97,2019-20,School,Lewis,Capital Region ESD 113,Adna School District,Adna Middle/High School,11th Grade,66,34.0,NaN,...,63.0,63,63.0,2020-02-11T00:00:00.000,100004.0,21226.0,100011.0,2441.0,101835.0,P
98,2019-20,School,Lewis,Capital Region ESD 113,Adna School District,Adna Middle/High School,12th Grade,56,30.0,NaN,...,51.0,54,52.0,2020-02-11T00:00:00.000,100004.0,21226.0,100011.0,2441.0,101835.0,P


In [65]:
wa_temp = wa_enrollment_data.loc[wa_enrollment_data['county'] != 'Multiple'].copy()
wa_temp.fillna(0, inplace=True)
wa_temp.groupby(wa_temp['county']).sum()


,all_students,female,gender_x,male,american_indian_alaskan_native,asian,black_african_american,hispanic_latino_of_any_race,native_hawaiian_other_pacific,two_or_more_races,...,non_migrant,non_military_parent,non_mobile,non_section_504,students_without_disabilities,esdorganizationid,districtcode,districtorganizationid,schoolcode,schoolorganizationid
county,,,,,,,,,,,,,,,,,,,,,
Adams,15,10.0,0.0,5.0,0.0,0.0,0.0,1.0,0.0,1.0,...,15,15,15.0,15,14.0,500005.0,5610.0,500105.0,15710.0,501645.0
Asotin,1250,626.0,0.0,624.0,4.0,14.0,2.0,76.0,4.0,70.0,...,1250,1238,1214.0,1226,1056.0,1600112.0,38720.0,1600240.0,39528.0,1605656.0
Clark,26802,13158.0,4.0,13640.0,118.0,642.0,270.0,2794.0,118.0,1450.0,...,26780,26556,25708.0,25892,22770.0,13300399.0,813827.0,13302394.0,497961.0,13541788.0
Grays Harbor,7064,3586.0,0.0,3478.0,174.0,104.0,68.0,2318.0,8.0,584.0,...,6914,6996,6624.0,6768,5800.0,7200288.0,1008360.0,7200720.0,246518.0,7313215.0
King,79748,38424.0,16.0,41308.0,496.0,21912.0,4346.0,16670.0,1774.0,7564.0,...,79708,79224,76320.0,76836,71538.0,38031240.0,6616921.0,38037579.0,1415182.0,38848722.0
Kitsap,7800,3746.0,2.0,4052.0,36.0,238.0,48.0,640.0,12.0,716.0,...,7800,7642,7674.0,6686,6804.0,6800408.0,1244604.0,6801156.0,180599.0,6949548.0
Lewis,1286,646.0,6.0,634.0,12.0,8.0,12.0,98.0,0.0,26.0,...,1286,1268,1248.0,1264,1150.0,1600064.0,339616.0,1600176.0,37344.0,1629352.0
Lincoln,260,134.0,0.0,126.0,0.0,2.0,0.0,16.0,0.0,26.0,...,260,260,252.0,260,228.0,1100011.0,242187.0,1100132.0,31460.0,1120735.0
Pierce,888,198.0,0.0,690.0,22.0,18.0,122.0,152.0,8.0,52.0,...,888,884,646.0,888,888.0,0.0,111724.0,416880.0,23800.0,416884.0


In [83]:
wa_temp['STATE'] = 'WA'

wa_clean = wa_temp[['county',
                    'american_indian_alaskan_native',
                    'asian',
                    'black_african_american',
                    'hispanic_latino_of_any_race',
                    'native_hawaiian_other_pacific',
                    'two_or_more_races',
                    'white']].copy()

wa_clean = wa_clean.groupby(wa_clean['county']).sum()
wa_clean

,american_indian_alaskan_native,asian,black_african_american,hispanic_latino_of_any_race,native_hawaiian_other_pacific,two_or_more_races,white
county,,,,,,,
Adams,0.0,0.0,0.0,1.0,0.0,1.0,13.0
Asotin,4.0,14.0,2.0,76.0,4.0,70.0,1080.0
Clark,118.0,642.0,270.0,2794.0,118.0,1450.0,21408.0
Grays Harbor,174.0,104.0,68.0,2318.0,8.0,584.0,3808.0
King,496.0,21912.0,4346.0,16670.0,1774.0,7564.0,26986.0
Kitsap,36.0,238.0,48.0,640.0,12.0,716.0,6110.0
Lewis,12.0,8.0,12.0,98.0,0.0,26.0,1130.0
Lincoln,0.0,2.0,0.0,16.0,0.0,26.0,216.0
Pierce,22.0,18.0,122.0,152.0,8.0,52.0,514.0


In [84]:

wa_clean.rename(
  columns={'county':'COUNTY',
 'american_indian_alaskan_native':'AMERICAN_INDIAN',
 'native_hawaiian_other_pacific':'PACIFIC_ISLANDER',
 'hispanic_latino_of_any_race':'LATINO',
 'black_african_american':'AFRICAN_AMERICAN',
 '2019-20 White ':'WHITE',
 'two_or_more_races':'OTHER/MULTI',
 'asian':'ASIAN',
  },
  inplace=True
)
wa_clean

,AMERICAN_INDIAN,ASIAN,AFRICAN_AMERICAN,LATINO,PACIFIC_ISLANDER,OTHER/MULTI,white
county,,,,,,,
Adams,0.0,0.0,0.0,1.0,0.0,1.0,13.0
Asotin,4.0,14.0,2.0,76.0,4.0,70.0,1080.0
Clark,118.0,642.0,270.0,2794.0,118.0,1450.0,21408.0
Grays Harbor,174.0,104.0,68.0,2318.0,8.0,584.0,3808.0
King,496.0,21912.0,4346.0,16670.0,1774.0,7564.0,26986.0
Kitsap,36.0,238.0,48.0,640.0,12.0,716.0,6110.0
Lewis,12.0,8.0,12.0,98.0,0.0,26.0,1130.0
Lincoln,0.0,2.0,0.0,16.0,0.0,26.0,216.0
Pierce,22.0,18.0,122.0,152.0,8.0,52.0,514.0


In [89]:
wa_clean['STATE'] = 'WA'
wa_clean

,AMERICAN_INDIAN,ASIAN,AFRICAN_AMERICAN,LATINO,PACIFIC_ISLANDER,OTHER/MULTI,white,STATE
county,,,,,,,,
Adams,0.0,0.0,0.0,1.0,0.0,1.0,13.0,WA
Asotin,4.0,14.0,2.0,76.0,4.0,70.0,1080.0,WA
Clark,118.0,642.0,270.0,2794.0,118.0,1450.0,21408.0,WA
Grays Harbor,174.0,104.0,68.0,2318.0,8.0,584.0,3808.0,WA
King,496.0,21912.0,4346.0,16670.0,1774.0,7564.0,26986.0,WA
Kitsap,36.0,238.0,48.0,640.0,12.0,716.0,6110.0,WA
Lewis,12.0,8.0,12.0,98.0,0.0,26.0,1130.0,WA
Lincoln,0.0,2.0,0.0,16.0,0.0,26.0,216.0,WA
Pierce,22.0,18.0,122.0,152.0,8.0,52.0,514.0,WA


In [92]:
wa_clean['AMERICAN_INDIAN'] = round(wa_clean['AMERICAN_INDIAN'],1)
wa_clean

,AMERICAN_INDIAN,ASIAN,AFRICAN_AMERICAN,LATINO,PACIFIC_ISLANDER,OTHER/MULTI,white,STATE
county,,,,,,,,
Adams,0.0,0.0,0.0,1.0,0.0,1.0,13.0,WA
Asotin,4.0,14.0,2.0,76.0,4.0,70.0,1080.0,WA
Clark,118.0,642.0,270.0,2794.0,118.0,1450.0,21408.0,WA
Grays Harbor,174.0,104.0,68.0,2318.0,8.0,584.0,3808.0,WA
King,496.0,21912.0,4346.0,16670.0,1774.0,7564.0,26986.0,WA
Kitsap,36.0,238.0,48.0,640.0,12.0,716.0,6110.0,WA
Lewis,12.0,8.0,12.0,98.0,0.0,26.0,1130.0,WA
Lincoln,0.0,2.0,0.0,16.0,0.0,26.0,216.0,WA
Pierce,22.0,18.0,122.0,152.0,8.0,52.0,514.0,WA


In [7]:
rds_connection_string = "postgres:postgres@localhost:5432/ELT_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [8]:
#Check for tables
engine.table_names()

[]